In [ ]:
#Check README for req/resp.Login to controller
import prisma_sase
import time
import pandas as pd
import termtables as tt
from time import sleep

PRISMASASE_CLIENT_ID = "xxxxxxxxxxxxx"
PRISMASASE_CLIENT_SECRET = "xxxxxxxx"
 
tsgid = "xxxxxxxx"
sdk = prisma_sase.API(controller="https://api.sase.paloaltonetworks.com", ssl_verify=False)
sdk.set_debug(3)
sdk.interactive.login_secret(client_id=PRISMASASE_CLIENT_ID, client_secret=PRISMASASE_CLIENT_SECRET, tsg_id=tsgid) 
#sdk.remove_header('X-PANW-Region')

In [ ]:
#Case 1:Fetch the Compute Location-SPN mapping and fetch the RN Site Status
url= "https://pa-us01.api.prismaaccess.com/api/sase/v2.0/resource/custom/query/remotenetworks/rn_list"
header = {
           "prisma-tenant": tsgid
        }
sdk._session.headers.update(header)


data = {
    "properties": [
    {
      "property": "edge_location_display_name",
      "alias": "location_name"
    },
    {
      "property": "site_name"
    },
    {
      "property": "ipsec_node"
    },
    {
      "property": "site_state_name"
    }
  ],
   
  "filter": {
    "operator": "AND",
    "rules": [
      {
        "property": "event_time",
        "operator": "last_n_days",
        "values": [30]
      },
      { 
        "property": "node_type",
        "operator": "in",
        "values": [48]
      }
    ]
    }
    
}

resp = sdk.rest_call(url=url, data=data, method="POST")
#print(resp)
try:
    prisma_sase.jd_detailed(resp)
except:
    print("No data found.")
    exit(0)

In [ ]:
#Case 2: Fetch the RN Location status
url= "https://pa-us01.api.prismaaccess.com/api/sase/v2.0/resource/query/edge_location_current_status"
header = {
           "prisma-tenant": tsgid
        }
sdk._session.headers.update(header)


data = {
  "properties": [
    {
      "property": "rn_state_instance"
    },
    {
      "property": "edge_location_display_name"
    }
  ],

   "filter": {
    "rules": [
      {
        "property": "rn_state_instance",
        "operator": "in",
        "values": [0, 1, 2]
      }
    ]
  },
  "count": 100
   
}

resp = sdk.rest_call(url=url, data=data, method="POST")
#print(resp)
try:
    prisma_sase.jd_detailed(resp)
except:
    print("No data found.")
    exit(0)


In [ ]:
#Case 3: Tunnel Status ,SPN throughput, SPN service IP (instance_public_ip)
url= "https://pa-us01.api.prismaaccess.com/api/sase/v2.0/resource/custom/query/tunnels/tunnel_list"
header = {
           "prisma-tenant": tsgid
        }
sdk._session.headers.update(header)


data = {
 "filter": {
    "operator": "AND",
    "rules": [
      {
        "property": "event_time",
        "operator": "last_n_days",
        "values": [30]
      },
      {
        "property": "tunnel_state",
        "operator": "in",
        "values": [1]
      },
      {
        "property": "node_type",
        "operator": "in",
        "values": [48]
      }
    ]
  }
   
}

resp = sdk.rest_call(url=url, data=data, method="POST")
#print(resp)
try:
    prisma_sase.jd_detailed(resp)
except:
    print("No data found.")
    exit(0)


In [ ]:
#Case 4: Agg BW remaining in a compute location
url= "https://pa-us01.api.prismaaccess.com/api/sase/v2.0/resource/custom/query/remotenetworks/rn_bandwidth_allocated"
header = {
           "prisma-tenant": tsgid
        }
sdk._session.headers.update(header)


data = {
  
  "filter": {
    "rules": [
      {
        "property": "event_time",
        "operator": "last_n_days",
        "values": [90]
      },
      {
        "property": "edge_location_display_name",
        "operator": "not_equals",
        "values": ["no_data"]
      }
    ]
  }
   
}

resp = sdk.rest_call(url=url, data=data, method="POST")
#print(resp)
try:
    prisma_sase.jd_detailed(resp)
except:
    print("No data found.")
    exit(0)


In [ ]:
#Case 5: Fetching min,max,avg,90th percentile per SPN in the past 1 hour
url= "https://pa-us01.api.prismaaccess.com/api/sase/v2.0/resource/custom/query/remotenetworks/rn_spn_node_peak_bw_minute"
header = {
           "prisma-tenant": tsgid
        }
sdk._session.headers.update(header)

now = time.time() * 1000 #milliseconds
then = now - 1*60*60*1000 #milliseconds
t1,t2 = int(then), int(now)

data = {
    "filter":
        {
            "rules":
            [
                {"property":"event_time",
                "operator":"between",
                "values":[t1,t2]
                }
            ]
        },
    "histogram":
        {"property":"event_time",
        "enableEmptyInterval":True,
        "range":"minute",
        "value":"3"}
   
}

resp = sdk.rest_call(url=url, data=data, method="POST")
#print(resp)
try:
    prisma_sase.jd_detailed(resp)
except:
    print("No data found.")
    exit(0)

try:
    dataList = resp.json()["data"]
except:
    print("No data found")
    exit(0)

df1 = pd.DataFrame(dataList)
df = df1.dropna()
   
spns = df.spn_name.unique()
RList = []
Header = ["SPN Name", "Min Throughput", "Average Throughput","90th percentile Throughput", "Max Throughput"]
   
for spn in spns:
    tmp = df[df.spn_name == spn]
        
    valdict = {}
    valdict["spn"] = spn
    #valdict["count"] = tmp.tunnel_throughput_greatest.count()
    valdict["min"] = round(tmp.tunnel_throughput_greatest.min(),2)
    valdict["avg"] = round(tmp.tunnel_throughput_greatest.mean(),2)
    valdict["90"] = round(tmp.tunnel_throughput_greatest.quantile(.9),2)
    valdict["max"] = round(tmp.tunnel_throughput_greatest.max(),2)
       
    valdict_rec = [valdict["spn"], valdict["min"],valdict["avg"],valdict["90"],valdict["max"]]
     
        
    RList.append(valdict_rec)
    #print(RList)
  

table_string = tt.to_string(RList, Header, style=tt.styles.ascii_thin_double)
print(table_string)




In [ ]:
#Case 6: Fetch the min,max,avg,90th percentile BW utilization for the past 24 hours
url= "https://pa-us01.api.prismaaccess.com/api/sase/v2.0/resource/custom/query/remotenetworks/rn_spn_node_peak_bw_hour"
header = {
           "prisma-tenant": tsgid
        }
sdk._session.headers.update(header)

now = time.time() * 1000 #milliseconds
then = now - 24*60*60*1000 #milliseconds
t1,t2 = int(then), int(now)

data = {
    "filter":
        {
            "rules":
            [
                {"property":"event_time",
                "operator":"between",
                "values":[t1,t2]
                }
            ]
        },
    "histogram":
        {"property":"event_time",
        "enableEmptyInterval":True,
        "range":"hour",
        "value":"3"}
   
}

resp = sdk.rest_call(url=url, data=data, method="POST")
#print(resp)
try:
    prisma_sase.jd_detailed(resp)
except:
    print("No data found.")
    exit(0)


df1 = pd.DataFrame(dataList)
df = df1.dropna()
   
spns = df.spn_name.unique()
RList = []
Header = ["SPN Name", "Min Throughput", "Average Throughput","90th percentile Throughput", "Max Throughput"]
   
for spn in spns:
    tmp = df[df.spn_name == spn]
        
    valdict = {}
    valdict["spn"] = spn
    #valdict["count"] = tmp.tunnel_throughput_greatest.count()
    valdict["min"] = round(tmp.tunnel_throughput_greatest.min(),2)
    valdict["avg"] = round(tmp.tunnel_throughput_greatest.mean(),2)
    valdict["90"] = round(tmp.tunnel_throughput_greatest.quantile(.9),2)
    valdict["max"] = round(tmp.tunnel_throughput_greatest.max(),2)
       
    valdict_rec = [valdict["spn"], valdict["min"],valdict["avg"],valdict["90"],valdict["max"]]
     
        
    RList.append(valdict_rec)
    #print(RList)
  

table_string = tt.to_string(RList, Header, style=tt.styles.ascii_thin_double)
print(table_string)
